In [1]:
 # Создаем spark сессию
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '16G')
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', 'False')
    .master("local[*]")
    .getOrCreate()
)

In [3]:
# файл с оценками - user * item матрица
import os
import pyspark.sql.functions as sql_func

DATA_DIR = 'D:/Datasets/ml-latest/'
# Сначала посмотрим на общее распределение тегов
ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, 'ratings.csv'),
        header=True,
        inferSchema=True
    )
    .drop('timestamp')
    .cache()
)

In [ ]:
%%time
(
    ratings.alias("one")
    .join(ratings.alias("two"), "userId")
    # расстояние симметрично
    # поэтому считаем только одну сторону
    .where("one.movieId > two.movieId")
    .groupBy("one.movieId", "two.movieId")
    .agg(
        sql_func.sum(
            sql_func.col("one.rating") *
            sql_func.col("two.rating")
        ).alias("inner_product"),
        sql_func.count("userId").alias("watched_both")
    ).select(
        sql_func.col("one.movieId").alias("movieId1"),
        sql_func.col("one.movieId").alias("movieId2"),
        sql_func.col("watched_both"),
        sql_func.col("inner_product")
    )
    .write
    .mode("overwrite")
    .parquet(os.path.join(DATA_DIR, "half_cooccurrences.parquet"))
)